# ResNet50V2
### Modelo ResNet50V2 utilizando apenas a imagem

In [201]:
import numpy as np
import pandas as pd
from PIL import Image
import keras
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split
from keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt


## Rede

In [202]:
resnet_cnn = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224,224,3),
    classifier_activation='softmax'
)

In [203]:
print("trainable_weights:", len(resnet_cnn.trainable_weights))
print("non_trainable_weights:", len(resnet_cnn.non_trainable_weights))

resnet_cnn.trainable = False; print('------------')

print("trainable_weights:", len(resnet_cnn.trainable_weights))
print("non_trainable_weights:", len(resnet_cnn.non_trainable_weights))

trainable_weights: 172
non_trainable_weights: 98
------------
trainable_weights: 0
non_trainable_weights: 270


In [205]:
inputs = keras.Input(shape=(224, 224, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = preprocess_input(inputs)
x = resnet_cnn(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

## Dados

In [206]:
batch_size = 48
img_height = 224
img_width = 224
data_dir = 'dados/imgs/'

In [207]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 209085 files belonging to 2 classes.
Using 167268 files for training.


In [208]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 209085 files belonging to 2 classes.
Using 41817 files for validation.


## Treino

Pensar em utilizar uma camada de normalização
```
normalization_layer = tf.keras.layers.Rescaling(1./255)
```


```
keras.metrics.F1Score(average=None, threshold=None, name="f1_score", dtype=None)
```

In [209]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.AUC()])

In [210]:
model.fit(train_ds, epochs=3, validation_data=val_ds)

Epoch 1/3
2466/3485 ━━━━━━━━━━━━━━━━━━━━ 18:54 1s/step - auc_6: 0.5129 - loss: 1.4784

InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Input is empty.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_one_step_on_iterator_151199]

In [ ]:
model.save_weights('resnet_models/model_v2.weights.h5')

## Resultados

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
          'test/',
           shuffle=False,
           batch_size=batch_size,
           image_size=(224,224))

Found 89925 files belonging to 1 classes.


In [ ]:
results = model.predict(test_dataset)

2811/2811 ━━━━━━━━━━━━━━━━━━━━ 2284s 813ms/step


In [ ]:
import glob
voos = pd.Series(glob.glob('./dados/test/test/*'))
ids = pd.DataFrame(voos.str.split('.').str[1].str.split('\\').str[1])

In [ ]:
# ids[1] = results
ids[1] = (results > 70).astype(np.int8)

In [ ]:
ids = ids.rename({0: 'flightid', 1: 'espera'},axis=1)

In [ ]:
df_2 = pd.read_csv('dados/public.csv')
df_2 = df_2[df_2.espera.isnull()]

In [ ]:
ids_faltantes = set(df_2.flightid) - set(ids.flightid)

In [ ]:
df_append = pd.DataFrame(list(ids_faltantes))
df_append[1] = 0
df_append = df_append.rename({0: 'flightid', 1: 'espera'},axis=1)

In [ ]:
sub = pd.concat([ids,df_append],axis=0)

In [ ]:
sub.to_csv('submission_resnet.csv',index=False)